# Configuration

In [ ]:
# These are connection parameters, could be coming from other source, like 'getpass'

vantage_host = "***"
vantage_username = "***"
vantage_password = "***"
val_db = "val"
byom_db = "mldb"
logmech = "tdnego"

In [ ]:
# making a connection to ModelOps and making sure we can see some projects

from aoa import AoaClient, ProjectApi
import pprint

client = AoaClient(
    aoa_url="https://vmo.local/core",
    auth_mode="client_credentials",
    auth_client_id="modelops-cli",
    auth_client_secret="***",
    ssl_verify=False,
    auth_token_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/token"
    ),
    auth_device_auth_url=(
        "https://vmo.local/sso/realms/teradata/protocol/openid-connect/auth/device"
    ),
)

project_api = ProjectApi(aoa_client=client)

projects = list(project_api)
pprint.pprint(projects)

# Saving a connection using API

In [ ]:
client.project_id = "23e1df4b-b630-47a1-ab80-7ad5385fcd8d"

personal_connection_payload = {
    "name": "MyPersonalConnection",
    "description": "Service Account Personal Connection",
    "metadata": {
        "host": vantage_host,
        "log_mech": "TDNEGO",
        "database": "",
        "valDb": val_db,
        "byomDb": byom_db,
    },
    "credentials": {
        "username": vantage_username,
        "password": vantage_password,
        "credsEncrypted": False,
    },
    "personal": True,
}

service_connection_payload = {
    "name": "MyServiceConnection",
    "description": "Service Account Service Connection",
    "metadata": {
        "host": vantage_host,
        "log_mech": "TDNEGO",
        "database": "",
        "valDb": val_db,
        "byomDb": byom_db,
    },
    "credentials": {
        "username": vantage_username,
        "password": vantage_password,
        "credsEncrypted": False,
    },
    "personal": False,
}

In [ ]:
from aoa import DatasetConnectionApi

connection_api = DatasetConnectionApi(aoa_client=client)

In [ ]:
connection_api.save(dataset_connection=personal_connection_payload)

In [ ]:
connection_api.save(dataset_connection=service_connection_payload)

# Validating a connection using API

In [ ]:
connections = list(connection_api)
connection = connections[0]
# This attribute needs to be added because API doesn't return decrypted credentials
connection["credentials"]["credsEncrypted"] = True
connection

In [ ]:
from requests.exceptions import HTTPError

try:
    result = client.post_request(
        path="/api/datasetConnections/validate",
        header_params=connection_api._get_header_params(),
        query_params={},
        body=connection,
    )
except HTTPError as err:
    print(err)
else:
    print(result)